In [1]:
# HIDDEN
from ipywidgets import widgets, interact, Dropdown,SelectMultiple, Checkbox,VBox,FloatSlider,Button
from IPython.display import display
#https://stackoverflow.com/questions/18398929/passing-multiple-variables-from-dropdowns-using-django-python
#https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
#https://medium.com/@jdchipox/how-to-interact-with-jupyter-33a98686f24e
#https://stackoverflow.com/questions/26352555/how-does-one-get-widget-values-with-a-button-in-ipython
#https://elc.github.io/posts/embed-interactive-notebooks/

In [2]:
# HIDDEN 
import numpy as np
import pandas as pd
import re
import os
from IPython import get_ipython
ipython = get_ipython()

In [3]:
# HIDDEN
style = {'description_width': 'initial'}
mnths = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
cntry = np.load('trandict.npy',allow_pickle='TRUE').item()
cntry = {v:k for k,v in cntry.items()}

In [4]:
# HIDDEN
listofcheckboxes = {'no preferences':[0],'fruits':['FAT'],'vegetables':['DGR','DGC'],
     'legumes':['DBR','DBC','DFR','DFC'],'grains':['AC','AC_wg','DAR','DAC',
     'AI','AI_wg','AA','AA_wg','AF','AF_wg','AD','AD_wg','AC','AC_wg'],'nuts_seeds':
     ['GAT'],'milk':['BAE','BAH','BAK','BLS','BLM','BLH','BLF','BN'],
     'vegetarianproducts':['BAV','BNV','VEG'],
     'fish':['JAC', 'JAR', 'JCC', 'JCR', 'JKC', 'JKR', 'JMC', 'JMR', 'JRC'],
    'meat':['MBGR','MBGC','MACR','MACC','MAER','MECC','MECR','MEER','MEEC','MAIC','MAIR','MAE',\
    'MAIR','MAIC','MCOR','MCOC','MCAR','MCAC','MAGR','MAGC','MAHR','MAHC',
    'MI','MAAC','MAAR'],'eggs_oil':['CA','OA', 'OC', 'OE','BTM','BJC']}

In [5]:
# HIDDEN
def joinlists_new(widgetname):
    global selectedfoods
    rs1 = [widgetname.description for widgetname in widgetname.children[1].children if widgetname.value]
    rs2 = [widgetname.value for widgetname in widgetname.children[2].children if widgetname.value]
    food = list(zip(rs1,rs2))  
    try:
        food
        for j in food:
            selectedfoods[j[0].split(',')[0]] = {}#'names'
            selectedfoods[j[0].split(',')[0]]['names']=j[0]
            selectedfoods[j[0].split(',')[0]]['mass']=j[1]
    except NameError:
        pass   
    return #selectedfoods

In [6]:
# HIDDEN
gender={'female':1,'male':0}
preferences = {'vegetarian':0,'vegan':1,'none':2}
indicator = {'climate change':'GHG','biodiversity loss':'BIO','both':'any'}
activity = {'sedentary':0,'moderate':1,'vigorous':2}
supp = {'yes':'yes_sup','no':'no_sup'}
weighting = {'climate change':0,'biodiversity loss':0,'both':0.5}
def weighting(impact):
    if impact == 'GHG' or 'BIO': w = 0
    elif impact == 'both':w = 0.5
    else: w = 0
    return w

# Choose your country and the month for which you'd like a diet

In [7]:
# HIDDEN
var1 = widgets.Dropdown(options = cntry.keys(),value = 'Switzerland')
display(var1)
var2 = widgets.Dropdown(options = mnths.keys(),value='aug')
display(var2)

Dropdown(index=26, options=('Aruba', 'Angola', 'Anguilla', 'Albania', 'Andorra', 'Argentina', 'American Samoa'…

Dropdown(index=7, options=('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec')…

# Please answer the following questions to personalize the diet to your needs

In [8]:
#HIDDEN
personweight = widgets.IntText(description='Weight (kg):',value=60)
display(personweight)
imp = widgets.Dropdown(options=indicator.values(),description='Impact')
display(imp)
gen = widgets.Dropdown(options=gender.keys(),description='Gender:')
display(gen)
height = widgets.IntText(description='Height (cm):',value=165)
display(height)
age = widgets.IntText(description='Age (yrs):',value=40)
display(age)
act = widgets.Dropdown(options=activity.keys(),description='Activity Level:',value='moderate')
display(act)
sup = widgets.Dropdown(options=supp.keys(),description='Include supplements?',style=style)
display(sup)
days = widgets.BoundedIntText(description='Number of Days for the Diet:',style=style,value=7)
display(days)
diettype = widgets.Dropdown(options=preferences.keys(),description='Dietary Preferences:',style=style,value='none')
display(diettype)


IntText(value=60, description='Weight (kg):')

Dropdown(description='Impact', options=('GHG', 'BIO', 'any'), value='GHG')

Dropdown(description='Gender:', options=('female', 'male'), value='female')

IntText(value=165, description='Height (cm):')

IntText(value=40, description='Age (yrs):')

Dropdown(description='Activity Level:', index=1, options=('sedentary', 'moderate', 'vigorous'), value='moderat…

Dropdown(description='Include supplements?', options=('yes', 'no'), style=DescriptionStyle(description_width='…

BoundedIntText(value=7, description='Number of Days for the Diet:', style=DescriptionStyle(description_width='…

Dropdown(description='Dietary Preferences:', index=2, options=('vegetarian', 'vegan', 'none'), style=Descripti…

In [9]:
# HIDDEN
## ADD A PROGRESS BAR
runcode2 = widgets.Button(description='Build the database')
output2 = widgets.Output()
def test(b):
    global df_foods
    with output2:
        print ('I want a diet for:',var1.value, '\nfor the month of:',var2.value)
        print ('This can take awhile.')
        ipython.magic('run Build_Optimization_Database.ipynb')
        dftest2 = calc_impacts(cntry[var1.value],var2.value)
        ipython.magic('run Foods_to_Include.ipynb')
        df_foods = editdf(dftest2,imp.value,diettype.value,supp[sup.value])
        print (df_foods.head(1))
        return df_foods

In [10]:
# HIDDEN
display(runcode2,output2)
runcode2.on_click(test)

Button(description='Build the database', style=ButtonStyle())

Output()

# Are there certain foods you must have?

In [ ]:
# HIDDEN
selectedfoods = {}
#Generate a dummy list
#Allfileslist = list(zip(df_foods['Food Name'],df_foods['Group']))
#Search box + generate ALL the checkboxes, this seems to be the bottleneck
search_widget = widgets.Dropdown(options = listofcheckboxes.keys(),layout={'width': 'initial'})
experiments = {}
experiments_mass = {}
options_widget = widgets.VBox(layout={'overflow': 'auto'})
mass_widget = widgets.VBox(layout={'overflow': 'auto'})
button_wid = widgets.Button(description='Save Choices')
but_out = widgets.Output()
def save(b):
    with but_out:
        joinlists_new(multi_select)
button_wid.on_click(save) 

In [29]:
# HIDDEN
selectedfoods = {}
#Generate a dummy list
#Allfileslist = list(zip(df_foods['Food Name'],df_foods['Group']))
#Search box + generate ALL the checkboxes, this seems to be the bottleneck
search_widget = widgets.Dropdown(options = listofcheckboxes.keys(),layout={'width': 'initial'})
experiments = {}
experiments_mass = {}
options_widget = widgets.VBox(layout={'overflow': 'auto'})
mass_widget = widgets.VBox(layout={'overflow': 'auto'})
button_wid = widgets.Button(description='Save Choices')
but_out = widgets.Output()
def save(b):
    with but_out:
        joinlists_new(multi_select)
button_wid.on_click(save) 

# HIDDEN
def whentextischanged (change):
    """Dynamically update the widget experiments"""
    search_input = change['new']
    if search_input == 'no preferences':
        # Reset search field, default to last 9 experiments
        new_options = [widgets.Checkbox(description='Proceed with optimization', value=True)]
        new_options_mass = [widgets.Checkbox(description='Proceed with optimization', value=True)]
    else:
        # Filter by search
        Allfileslist = list(zip(df_foods['Food Name'],df_foods['Group']))
        close_matches = [x[0] for x in Allfileslist if x[1] in(listofcheckboxes[search_widget.value])][0:5]
        for name in close_matches:
            if name not in experiments:
                experiments[name] = widgets.Checkbox(description=name, value=False,style={'description_width': 'initial'})
                experiments_mass[name] = widgets.IntSlider(description='Mass (grams)', min=0,max=500,step=50,value=0)
        ### magic code that generates only the match boxes ###
        new_options = [experiments[eachfilename] for eachfilename in close_matches]
        new_options_mass = [experiments_mass[eachfilename] for eachfilename in close_matches]
    options_widget.children = new_options
    mass_widget.children = new_options_mass

In [30]:
multi_select = widgets.HBox([search_widget, options_widget,mass_widget,button_wid])
search_widget.observe(whentextischanged, names='value')
display(multi_select)

In [28]:
selectedfoods

{'apples': {'names': 'apples, eating, raw, flesh and skin, weighed with core_FAT',
  'mass': 150}}

In [ ]:
# HIDDEN
runcodeprint = widgets.Button(description='Show Must Haves')
outputprint = widgets.Output()

In [ ]:
selectedfoods

In [ ]:
def printfoods(b):
    global selectedfoods
    with outputprint:
        print (selectedfoods)
display(runcodeprint,outputprint)
runcodeprint.on_click(printfoods)

In [ ]:
# HIDDEN
runcode5 = widgets.Button(description='Click Here to Build your Low Impact and Healthful Diet')
output5 = widgets.Output()
def run_tool(b):
    global df_opt
    with output5:       
        ipython.magic('run Optimization_Tool_Online.ipynb')
        print ('This can take some time.')
        df_opt = runopt(df_foods,imp.value,diettype.value,supp[sup.value],gender[gen.value],personweight.value,\
                          height.value,age.value,act.value,days.value,2,selectedfoods)
        return df_opt[(df_opt['Grams']!=0)&(df_opt['Grams'].notnull())]
display(runcode5,output5)
runcode5.on_click(run_tool)